# Resampling

In [42]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
summarize ,
poly)
from sklearn.model_selection import train_test_split
from functools import partial
from sklearn.model_selection import \
(cross_validate ,
KFold ,
ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

### Data loading and splitting

In [12]:
Auto = load_data('Auto')
# int(Auto.shape[0] / 2) can replace with 196
Auto_train,Auto_valid = train_test_split(Auto, test_size = int(Auto.shape[0] / 2), random_state=0)


### Training the model with training data split

In [24]:
hp_mm = MS(['horsepower'])
X_train =hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model =sm.OLS(y_train, X_train)
results = model.fit()

### Evaluate the model matrix for this model created using the validation data set

In [25]:
X_valid = hp_mm.transform(Auto_valid)
y_valid =Auto_valid['mpg']
valid_pred = results.predict(X_valid)
np.mean((y_valid - valid_pred) ** 2)


23.61661706966988

#### Hence our estimate for the validation MSE of the linear regression fit is: 23.62.

### A function evalMSE() that takes a model string as well as a training and test set and returns the MSE on the test set

In [33]:
def evalMSE(terms ,response , train , test):
    mm = MS(terms)
    X_train = mm.fit_transform(train)
    y_train = train[response]
    X_test = mm.transform(test)
    y_test = test[response]
    results = sm.OLS(y_train , X_train).fit()
    test_pred = results.predict(X_test)
    return np.mean((y_test - test_pred)**2)

MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)], 'mpg', Auto_train, Auto_valid)
MSE

array([23.61661707, 18.76303135, 18.79694163])

### If we choose a different training/validation split instead

In [50]:
Auto_train , Auto_valid = train_test_split(Auto,test_size=196, random_state=3)
MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)], 'mpg', Auto_train , Auto_valid)
MSE

array([20.75540796, 16.94510676, 16.97437833])

Using this split of the observations into a training set and a validation
set, we find that the validation set error rates for the models with linear,
quadratic, and cubic terms are 20.76, 16.95, and 16.97, respectively.

### Cross-Validation

In [55]:
hp_model =sklearn_sm(sm.OLS, MS(['horsepower']))
X,Y =Auto.drop(columns=['mpg']), Auto['mpg']
cv_results = cross_validate(hp_model, X,Y, cv=Auto.shape[0])
cv_errors = np.mean(cv_results['test_score'])
cv_errors

24.231513517929212

### To automate the process

### LOOCV Validation

In [57]:
cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
M = sklearn_sm(sm.OLS)
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M, X, Y, cv=Auto.shape[0])
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.23151352, 19.24821312, 19.33498406, 19.42443031, 19.03320428])

Above we introduced the outer() method of the np.power() function.
The outer() method is applied to an operation that has two arguments such as add(), min(), or power().

In [58]:
A = np.array([3, 5, 9])
B = np.array([2, 4])
np.add.outer(A, B)

array([[ 5,  7],
       [ 7,  9],
       [11, 13]])

### KFold

In [59]:
cv_error = np.zeros(5)
cv = KFold(n_splits=10,
shuffle=True,
random_state=0) # use same splits for each degree
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M, X, Y, cv=cv)
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.20766449, 19.18533142, 19.27626666, 19.47848404, 19.13722016])

### ShuffleSplit

In [60]:
validation = ShuffleSplit(n_splits=1,
test_size=196,
random_state=0)
results = cross_validate(hp_model ,
Auto.drop(['mpg'], axis=1),
Auto['mpg'],
cv=validation);
results['test_score']

array([23.61661707])

In [61]:
validation = ShuffleSplit(n_splits=10,
test_size=196,
random_state=0)
results = cross_validate(hp_model ,
Auto.drop(['mpg'], axis=1),
Auto['mpg'],
cv=validation)
results['test_score'].mean(), results['test_score'].std()

(23.802232661034164, 1.4218450941091847)

### The Bootstrap

In [64]:
Portfolio = load_data('Portfolio')
Portfolio = load_data('Portfolio')
def alpha_func(D, idx):
    cov_ = np.cov(D[['X','Y']].loc[idx], rowvar=False)
    return ((cov_[1,1] - cov_[0,1]) /
(cov_[0,0]+cov_[1,1]-2*cov_[0,1]))

In [65]:
alpha_func(Portfolio , range(100))

0.57583207459283

### Next we randomly select 100 observations from range(100)

In [67]:
rng = np.random.default_rng(0)
alpha_func(Portfolio , rng.choice(100, 100, replace=True))

0.6074452469619004